In [12]:
import numpy as np

"""
两遍扫描法 获取连通域
"""

'\n两遍扫描法 获取连通域\n'

In [13]:
demo_list = [
    0, 0, 1, 0, 0, 1, 0,
    1, 1, 1, 0, 1, 1, 1,
    0, 0, 1, 0, 0, 1, 0,
    0, 1, 1, 0, 1, 1, 0,
]
_id = 1

In [14]:
array = np.array(demo_list).reshape(4, 7)
array_shape = array.shape
print(array_shape)
array

(4, 7)


array([[0, 0, 1, 0, 0, 1, 0],
       [1, 1, 1, 0, 1, 1, 1],
       [0, 0, 1, 0, 0, 1, 0],
       [0, 1, 1, 0, 1, 1, 0]])

In [15]:
# 创建一个同行同列的array, 用于存放 id
id_array = np.zeros(array_shape, dtype=int)
id_array

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]])

In [16]:
# 创建一个 dict 用于存放有相邻关系的id
id_adjacent = {}

In [17]:
# 第一次遍历
for i in range(array_shape[0]):
    for j in range(array_shape[1]):
        # 注意临界条件就行
        # 第一行只看左边是否相同
        # 第一列只看上面是否相同
        # 第一行第一列直接标记
        if i == 0 and j == 0:  # 第一行第一列
            id_array[i][j] = _id
            id_adjacent[_id] = set()  # 新建集合
            _id += 1
        elif i == 0:  # 第一行非第一个, 只往左边找
            if array[i][j] == array[i][j-1]:  # 与左边相同, 则连通
                id_array[i][j] = id_array[i][j-1]
            else:  # 不相同
                id_array[i][j] = _id
                id_adjacent[_id] = set()
                _id += 1
        elif j == 0:  # 第一列非第一个, 只往上面找
            if array[i][j] == array[i-1][j]:  # 与上面相同, 则连通
                id_array[i][j] = id_array[i-1][j]
            else:
                id_array[i][j] = _id
                id_adjacent[_id] = set()
                _id += 1
        else:  # 先往上面找, 再往左边找
            if array[i][j] == array[i-1][j] and array[i][j] != array[i][j-1]:  # 只与上面相同
                id_array[i][j] = id_array[i-1][j]
            elif array[i][j] == array[i][j-1] and array[i][j] != array[i-1][j]:  # 只与左边相同
                id_array[i][j] = id_array[i][j-1]
            elif array[i][j] != array[i][j-1] and array[i][j] != array[i-1][j]:  # 上面、左边都不相同
                id_array[i][j] = _id
                id_adjacent[_id] = set()
                _id += 1
            else:  # 上面、左边都相同
                id_array[i][j] = id_array[i-1][j]  # 优先用上面的id
                if id_array[i][j-1] != id_array[i-1][j]:
                    id_adjacent[id_array[i-1][j]].add(id_array[i][j-1])

In [18]:
array

array([[0, 0, 1, 0, 0, 1, 0],
       [1, 1, 1, 0, 1, 1, 1],
       [0, 0, 1, 0, 0, 1, 0],
       [0, 1, 1, 0, 1, 1, 0]])

In [19]:
id_array

array([[ 1,  1,  2,  3,  3,  4,  5],
       [ 6,  6,  2,  3,  7,  4,  4],
       [ 8,  8,  2,  3,  3,  4,  9],
       [ 8, 10,  2,  3, 11,  4,  9]])

In [20]:
id_adjacent

{1: set(),
 2: {6, 10},
 3: set(),
 4: {7, 11},
 5: set(),
 6: set(),
 7: set(),
 8: set(),
 9: set(),
 10: set(),
 11: set()}

In [21]:
# 处理 id dict
for i in range(_id-1, 0, -1):  # 区间: (0, _id-1], 不过是倒序的
    merge = False
    for j in range(i-1, 0, -1):  # 区间: (0, i-1], 倒序
        if i in id_adjacent[j] or not id_adjacent[j].isdisjoint(id_adjacent[i]):
            id_adjacent[j].update(id_adjacent[i])  # 合并集合
            merge = True
    if merge:
        del id_adjacent[i]  # 有合并情况则删除当前集合
id_adjacent

{1: set(), 2: {6, 10}, 3: set(), 4: {7, 11}, 5: set(), 8: set(), 9: set()}

In [22]:
# 第二遍扫描
for i in range(array_shape[0]):
    for j in range(array_shape[1]):
        if id_array[i][j] in id_adjacent:
            continue
        for k in id_adjacent:
            if id_array[i][j] in id_adjacent[k]:
                id_array[i][j] = k
                break

array([[1, 1, 2, 3, 3, 4, 5],
       [2, 2, 2, 3, 4, 4, 4],
       [8, 8, 2, 3, 3, 4, 9],
       [8, 2, 2, 3, 4, 4, 9]])

In [24]:
# 现在 id_array 里面就保存了连通信息, 相同id都是连通
id_array

array([[1, 1, 2, 3, 3, 4, 5],
       [2, 2, 2, 3, 4, 4, 4],
       [8, 8, 2, 3, 3, 4, 9],
       [8, 2, 2, 3, 4, 4, 9]])